# FromageModel Inference Notebook 
## 0. Imports & Config

In [1]:
import torch
import torch.nn as nn
import yaml
import random
import sys

In [2]:
sys.path.append('../fromage')
from model import Fromage, FromageModel
from experiment import Experiment
from data import MIMICDataset, cxr_image_transform
from utils import preprocess_report

In [3]:
ckpt_path = "../logs/checkpoints/untied_test/last.ckpt"
config_path = "../config/train-untied.yaml"
dataset_path = "../data/MIMIC_JPG.tsv"
img_path = "/datasets/mimic/cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/files"

## 1. Set device and config file

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open (config_path) as file:
    config = yaml.safe_load(file)

## 2. Load model and dataset

In [5]:
model = Experiment(config)
model = model.load_from_checkpoint(ckpt_path)
model = model.model 

RESIZE = 512
CENTER_CROP_SIZE = 480

transform = cxr_image_transform(RESIZE, CENTER_CROP_SIZE, train=True)
dataset = MIMICDataset(dataset_path, img_path, transform)

## 3. Get random example report from dataset

In [6]:
ex_idx = random.randint(0, len(dataset.img_paths) - 1)
ex_img_path, ex_report = dataset.img_paths[ex_idx], dataset.reports[ex_idx]
print(ex_report)

INDICATION: year old man with CNS lymphoma, cardiac history, cough, right lower chestrib pain on inspiration ro pneumonia vs rib fx. ro pneumonia vs rib fx. IMPRESSION: In comparison with the study of , allowing for slightly better inspiration, there is little change. Continued enlargement of the cardiac silhouette without vascular congestion, pleural effusion, or acute focal pneumonia. Substantial mass is again seen in the lower right neck displacing of the cervical trachea to the left.


## 4. Inference

In [7]:
with torch.inference_mode():
    model.eval()
    prompts = [ex_img_path, ex_report[:150]] 
    print(model.generate_for_images_and_texts(prompts))

 pneumonia.

SUMMARY: The patient is in a stable condition. The patient is in a stable condition. The patient is in a stable condition
